In [ ]:
import os
from os import path
from io import StringIO
import re
import pdfplumber
import pandas as pd

cwd_dir = path.join(os.getcwd())
root_dir = path.dirname(cwd_dir)

raw_dir = path.join(root_dir, 'raw')
db_dir = path.join(root_dir, 'database')
src_dir = path.join(root_dir, 'scripts')
tmpl_dir = path.join(root_dir, 'template')
dist_dir = path.join(root_dir, 'dist')

alipay_dir = path.join(raw_dir, 'alipay')
alipay_csv_dist = path.join(alipay_dir, 'alipay.csv')
wechat_dir = path.join(raw_dir, 'wechat')
wechat_csv_dist = path.join(dist_dir, 'wechat.csv')
cmb_credit_dir = path.join(raw_dir, 'cmb', 'credit')
cmb_credit_csv_dist = path.join(dist_dir, 'cmb_credit.csv')
cmb_debit_dir = path.join(raw_dir, 'cmb', 'debit')
cmb_debit_csv_dist = path.join(dist_dir, 'cmb_debit.csv')

In [ ]:
# extract cmb debit with ocr
cmb_debit_files = list(filter(lambda x: x.endswith('.pdf'), os.listdir(cmb_debit_dir)))

cmb_debit_page_0_bbox = [0, 240, 595, 750]
cmb_debit_page_x_bbox = [0, 30, 595, 760]
cmb_debit_columns = [
    'trans_time',
    'currency',
    'amount',
    'balance',
    'kind',
    'target'
]


def cmb_debit_pdf_to_df(file: str) -> pd.DataFrame:
    ls = []
    with pdfplumber.open(file) as pdf:
        for pi, page in enumerate(pdf.pages):
            px = page.crop(bbox=cmb_debit_page_0_bbox if pi == 0 else cmb_debit_page_x_bbox)
            tx = px.extract_table(table_settings={
                'vertical_strategy': 'text',
                'horizontal_strategy': 'text',
                'text_keep_blank_chars': True
            })
            lsx = []
            for i, r in enumerate(tx):
                r = ['' if w is None or w.isspace() else w for w in r]
                if len(lsx) == 0 and not re.match(r'^\d{4}-\d{2}-\d{2}$', r[0]):
                    continue
                empty_counts = 0
                for w in r:
                    if w is None or w == '':
                        empty_counts += 1
                if empty_counts == len(r):
                    continue
                elif empty_counts >= 3 and lsx[-1] is not None:
                    last = lsx[-1]
                    for j in range(0, len(last)):
                        w = r[j]
                        last[j] = f'{last[j]} {w}'.strip()
                else:
                    lsx.append(r)
            ls.extend(lsx)
    return pd.DataFrame(
        ls,
        columns=cmb_debit_columns
    ).replace('/', '').apply(lambda x: x.str.strip())


def cmb_debit_pdf_batch_to_df() -> pd.DataFrame:
    return (pd.concat(
        [cmb_debit_pdf_to_df(path.join(cmb_debit_dir, f)) for f in cmb_debit_files],
        ignore_index=False
    ).sort_values(by=['trans_time'])
     .reset_index(drop=True))


cmb_debit_df = cmb_debit_pdf_batch_to_df()
cmb_debit_df.to_csv(cmb_debit_csv_dist, index_label='id', index=True)
cmb_debit_df

In [ ]:
# extract cmb credit with ocr
cmb_credit_files = list(filter(lambda x: x.endswith('.pdf'), os.listdir(cmb_credit_dir)))

cmb_credit_columns = ['sold_time', 'posted_time', 'desc', 'rmb_amount', 'card_no_snapshot', 'origin_amount']


def cmb_credit_pdf_to_df(file: str) -> pd.DataFrame:
    ls = []
    year = path.basename(file)[0:4]
    with pdfplumber.open(file) as pdf:
        for pi, page in enumerate(pdf.pages):
            px = page
            txs = px.extract_tables(table_settings={
                'vertical_strategy': 'lines',
                'horizontal_strategy': 'lines',
                'text_keep_blank_chars': True
            })
            for tx in txs:
                for r in tx:
                    r = [w.strip() if w else '' for w in r]
                    if len(r) < len(cmb_credit_columns):
                        continue
                    elif re.match(r'\d{2}/\d{2}', r[1]):
                        r[1] = f'{year}/{r[1]}'
                        if re.match(r'\d{2}/\d{2}', r[0]):
                            r[0] = f'{year}/{r[0]}'
                    else:
                        continue
                    ls.append(r)
    return pd.DataFrame(ls, columns=cmb_credit_columns).replace('/', '').apply(lambda x: x.str.strip())


def cmb_credit_pdf_batch_to_df() -> pd.DataFrame:
    return (pd.concat(
        [cmb_credit_pdf_to_df(path.join(cmb_credit_dir, f)) for f in cmb_credit_files],
        ignore_index=False
    ).sort_values(by=['posted_time', 'sold_time'])
     .reset_index(drop=True))


cmb_credit_df = cmb_credit_pdf_batch_to_df()
cmb_credit_df.to_csv(cmb_credit_csv_dist, index_label='id', index=True)
cmb_credit_df

In [ ]:
# extract wechat to df

wechat_files = list(filter(lambda x: x.endswith('.csv'), os.listdir(wechat_dir)))
wechat_columns = [
    'trans_time',
    'kind',
    'target',
    'goods',
    'io',
    'rmb_amount',
    'method',
    'status',
    'trans_id',
    'mall_trans_id',
    'desc'
]


def wechat_csv_to_df(file: str) -> pd.DataFrame:
    lines = []
    with open(file, encoding='utf-8') as f:
        lines = list(
            filter(
                lambda x: x != '' and not x.endswith(',,,,,,,,') and re.match(r'^\d{4}-\d{2}-\d{2}', x),
                [r.strip() for r in f.readlines()]
            ))
    return pd.read_csv(
        StringIO('\n'.join(lines)),
        names=wechat_columns,
        dtype=str,
        parse_dates=False
    ).apply(lambda x: x.str.strip()).replace('/', '')


def wechat_csv_batch_to_df() -> pd.DataFrame:
    return pd.concat(
        [
            wechat_csv_to_df(path.join(wechat_dir, f)) for f in wechat_files
        ],
        ignore_index=None
    ).sort_values(
        by='trans_time'
    ).reset_index(drop=True)


wechat_df = wechat_csv_batch_to_df()
wechat_df.to_csv(wechat_csv_dist, index_label='id', index=True)
wechat_df

In [ ]:
# extract alipay to df

alipay_files = list(filter(lambda x: x.endswith('.csv'), os.listdir(alipay_dir)))
alipay_columns = [
    'trans_id',
    'mall_trans_id',
    'trans_time',
    'pay_time',
    'updated_time',
    'source',
    'kind',
    'target',
    'goods',
    'rmb_amount',
    'io',
    'trans_status',
    'service_charge_rmb_amount',
    'refund_rmb_amount',
    'desc',
    'money_status'
]


def alipay_csv_to_df(file: str) -> pd.DataFrame:
    lines = []
    with open(file, encoding='utf-8') as f:
        lines = list(
            filter(
                lambda x: x != '' and re.match(r'^\d{6}', x),
                [r.strip().replace('\t', '') for r in f.readlines()]
            ))
    return (pd.read_csv(
        StringIO('\n'.join(lines)),
        names=alipay_columns + ['unknown1'],
        dtype=str,
        parse_dates=False
    ).apply(lambda x: x.str.strip())
            .drop(columns=['unknown1']))


def alipay_csv_batch_to_df() -> pd.DataFrame:
    return pd.concat(
        [
            alipay_csv_to_df(path.join(alipay_dir, f)) for f in alipay_files
        ],
        ignore_index=True
    ).sort_values(by=[
        'trans_time',
        'trans_id'
    ]).reset_index(drop=True)


alipay_df = alipay_csv_batch_to_df()
alipay_df.to_csv(alipay_csv_dist, index_label='id', index=True)
alipay_df